# Measuring overhead of Tensorflow for small ops

This notebook measures the overhead of tensorflow for large graphs of lightweight ops, as are common in TreeCat.

See Tensorflow issues [#120](https://github.com/tensorflow/tensorflow/issues/120) and [#5981](https://github.com/tensorflow/tensorflow/issues/5981) for prior art.

In [1]:
import tensorflow as tf
import time

In [2]:
M = 32    # Matrix rows.
N = 32    # Matrix cols.
T = 1000  # Time steps.
P = 10    # Parallel sequences.
with tf.Graph().as_default():
    a = tf.constant([[1.0] * N] * M, dtype=tf.float32)
    b = tf.constant([[-0.5] * N] * M, dtype=tf.float32)
    xs = [a] * P
    for i in range(P):
        for _ in range(T // 2):
            xs[i] += a
            xs[i] *= b
    x = tf.parallel_stack(xs)
    with tf.Session():
        start = time.time()
        x.eval()
        end = time.time()
print('{:0.3g}us / op'.format((end - start) / (P * T) * 1e6))

53.6us / op


In [25]:
with tf.Graph().as_default():
    probs = tf.constant([0.1, 0.2, 0.3, 0.4])
    logits = tf.log(probs)
    samples = tf.multinomial(logits[tf.newaxis, :], 10)[0]
    with tf.Session():
        print(samples.eval())

[3 3 3 2 2 1 2 3 2 3]


In [28]:
from collections import Counter
with tf.Graph().as_default():
    probs = tf.constant([0.1, 0.2, 0.3, 0.4])
    # The p/(1-p) formula is incorrect:
    # logits = tf.log(probs / (1.0 - probs))  # Incorrect.
    logits = tf.log(probs)  # Correct.
    samples = tf.multinomial(logits[tf.newaxis, :], 1000000)[0]
    with tf.Session():
        s = samples.eval()
print(sorted(Counter(s).items()))

[(0, 100090), (1, 200355), (2, 299947), (3, 399608)]
